### 一.基本的PageRank算法
PageRank算法是用于网页权重计算的算法，它本质就是一个马尔可夫模型，每个网页$Page(i),i=1,2,..,n$就是一个状态，状态的转移概率由该页面超链接的其他网页均摊，而PageRank值即是马尔可夫模型的平稳态概率分布，平稳态概率分布的每个分量即是对应的每个网页的PageRank值，比如如下的网页A、B、C、D的链接关系：  
![avatar](./source/12_pagerank_demo1.png)  

可以很方便的写出它的状态转移矩阵（A,B,C,D分别对应0,1,2,3）：   

$$
P=\begin{bmatrix}
0 & 1/2 & 1 & 0 \\ 
1/3 & 0 & 0 & 1/2 \\ 
1/3 & 0 & 0 & 1/2\\ 
1/3 & 1/2 & 0 & 0
\end{bmatrix}
$$   

下面，我们可以直接利用其马尔可夫模型求解其PageRank值

In [1]:
import os
os.chdir('../')
# from ml_models.pgm import SimpleMarkovModel
import numpy as np

In [2]:
"""
齐次时间、离散、有限状态、一阶马尔可夫链的实现
"""
import numpy as np


class SimpleMarkovModel(object):
    def __init__(self, status_num=None):
        # 初始状态向量
        self.pi = np.zeros(shape=(status_num, 1))
        # 状态转移概率矩阵
        self.P = np.zeros(shape=(status_num, status_num))

    def fit(self, x):
        """
        根据训练数据，统计计算初始状态向量以及状态转移概率矩阵
        :param x: x可以是单列表或者是列表的列表，比如[s1,s2,...,sn]或者[[s11,s12,...,s1m],[s21,s22,...,s2n],...],
                 计算初始状态向量的方式会有差异，单列表会统计所有所有状态作为初始状态分布，列表的列表会统计所有子列表开头
                 状态的分布
        :return:
        """
        if type(x[0]) == list:
            for clist in x:
                self.pi[clist[0]] += 1
                for cindex in range(0, len(clist) - 1):
                    self.P[clist[cindex + 1], clist[cindex]] += 1
        else:
            for index in range(0, len(x) - 1):
                self.pi[x[index]] += 1
                self.P[x[index + 1], x[index]] += 1
        # 归一化
        self.pi = self.pi / np.sum(self.pi)
        self.P = self.P / np.sum(self.P, axis=0)

    def predict_log_joint_prob(self, status_list):
        """
        计算联合概率的对数
        :param status_list:
        :return:
        """
        # 这里偷懒就不并行计算了...
        log_prob = np.log(self.pi[status_list[0], 0])
        for index in range(0, len(status_list) - 1):
            log_prob += np.log(self.P[status_list[index + 1], status_list[index]])
        return log_prob

    def predict_prob_distribution(self, time_steps=None, set_init_prob=None, set_prob_trans_matrix=None):
        """
        计算time_steps后的概率分布，允许通过set_init_prob和set_prob_trans_matrix设置初始概率分布和概率转移矩阵
        :param time_steps:
        :param set_init_prob:
        :param set_prob_trans_matrix:
        :return:
        """
        prob = self.pi if set_init_prob is None else set_init_prob
        trans_matrix = self.P if set_prob_trans_matrix is None else set_prob_trans_matrix
        for _ in range(0, time_steps):
            prob = trans_matrix.dot(prob)
        return prob

    def predict_next_step_prob_distribution(self, current_status=None):
        """
        预测下一时刻的状态分布
        :param current_status:
        :return:
        """
        return self.P[:, [current_status]]

    def predict_next_step_status(self, current_status=None):
        """
        预测下一个时刻最有可能的状态
        :param current_status:
        :return:
        """
        return np.argmax(self.predict_next_step_prob_distribution(current_status))

    def generate_status(self, step_times=10, stop_status=None, set_start_status=None, search_type="greedy", beam_num=5):
        """
        生成状态序列，包括greedy search和beam search两种方式
        :param step_times: 步长不超过 step_times
        :param stop_status: 中止状态列表
        :param set_start_status: 人为设置初始状态
        :param search_type: 搜索策略，包括greedy和beam
        :param beam_num: 只有在search_type="beam"时生效，保留前top个结果
        :return:
        """
        if stop_status is None:
            stop_status = []
        # 初始状态
        start_status = np.random.choice(len(self.pi.reshape(-1)),
                                        p=self.pi.reshape(-1)) if set_start_status is None else set_start_status
        if search_type == "greedy":
            # 贪婪搜索
            rst = [start_status]
            for _ in range(0, step_times):
                next_status = self.predict_next_step_status(current_status=start_status)
                rst.append(next_status)
                if next_status in stop_status:
                    break
        else:
            # beam search
            rst = [start_status]
            top_k_rst = [[start_status]]
            top_k_prob = [0.0]
            for _ in range(0, step_times):
                new_top_k_rst = []
                new_top_k_prob = []
                for k_index, k_rst in enumerate(top_k_rst):
                    k_rst_last_status = k_rst[-1]
                    # 获取前k大的idx
                    top_k_idx = self.P[:, k_rst_last_status].argsort()[::-1][0:beam_num]
                    for top_k_status in top_k_idx:
                        new_top_k_rst.append(k_rst + [top_k_status])
                        new_top_k_prob.append(top_k_prob[k_index] + np.log(1e-12+self.P[top_k_status, k_rst_last_status]))
                # 对所有的beam_num*beam_num个结果排序取前beam_num个结果
                top_rst_idx = np.asarray(new_top_k_prob).argsort()[::-1][0:beam_num]
                rst = new_top_k_rst[top_rst_idx[0]]
                # 更新
                top_k_rst = []
                top_k_prob = []
                for top_idx in top_rst_idx[:beam_num]:
                    if new_top_k_rst[top_idx][-1] in stop_status:
                        rst = new_top_k_rst[top_idx]
                        break
                    else:
                        top_k_rst.append(new_top_k_rst[top_idx])
                        top_k_prob.append(new_top_k_prob[top_idx])
        return rst


In [3]:
#定义初始概率
init_prob=np.ones(shape=(4,1))/4.0
#定义概率转移矩阵
trans_matrix=np.asarray([
    [0.,0.5,1.0,0],
    [1.0/3,0,0,1.0/2],
    [1.0/3,0,0,1.0/2],
    [1.0/3,1.0/2,0,0],
])

In [4]:
#求解pagerank：即通过足够长的step后的平稳态概率分布
smm=SimpleMarkovModel(status_num=4)
smm.predict_prob_distribution(set_init_prob=init_prob,set_prob_trans_matrix=trans_matrix,time_steps=10)

array([[0.33325195],
       [0.22224935],
       [0.22224935],
       [0.22224935]])

In [5]:
#校验：设置其他的step校验一下
smm.predict_prob_distribution(set_init_prob=init_prob,set_prob_trans_matrix=trans_matrix,time_steps=20)

array([[0.33333325],
       [0.22222225],
       [0.22222225],
       [0.22222225]])

### 二.带平滑项的PageRank算法
但是，实际情况往往会比较复杂，比如某些网页只进不出，如下图网页C就只进不出，这样会导致马尔可夫链无法收敛到一个稳定态   

![avatar](./source/12_pagerank_demo2.png)  

这时，它的概率转移矩阵为：   

$$
P=\begin{bmatrix}
0 & 1/2 & 0 & 0 \\ 
1/3 & 0 & 0 & 1/2 \\ 
1/3 & 0 & 0 & 1/2\\ 
1/3 & 1/2 & 0 & 0
\end{bmatrix}
$$   

我们再求解一下它的PageRank值

In [6]:
trans_matrix=np.asarray([
    [0.,0.5,0,0],
    [1.0/3,0,0,1.0/2],
    [1.0/3,0,0,1.0/2],
    [1.0/3,1.0/2,0,0],
])
smm.predict_prob_distribution(set_init_prob=init_prob,set_prob_trans_matrix=trans_matrix,time_steps=50)

array([[2.55407417e-08],
       [3.72237693e-08],
       [3.72237693e-08],
       [3.72237693e-08]])

几乎都逼近到0了，我们可以通过添加一个平滑项来解决这个问题，即：   

$$
A=dP+\frac{(1-d)}{n}E
$$  

这里，$0\leq d \leq 1$，$E$是元素均为1的方阵，$n$表示状态数，注意，这样对每一列求和后可能由某些列的概率概率值<1（比如我们上面问题中的第三列），所以对于每一步更新：   

$$
\pi_{t+1}=A\pi_t
$$  

都需要对$\pi_{t+1}$做一次归一化操作，一般取无穷范数作归一化，即$\pi_{t+1}$中绝对值最大的分量：   

$$
\pi_{t+1}=\frac{\pi_{t+1}}{\mid\mid \pi_{t+1}\mid\mid}
$$  

下面对PageRank算法做一个单独的实现：   

In [7]:
"""
PageRank算法实现，封装到ml_models.pgm
"""

class PageRank(object):
    def __init__(self, init_prob, trans_matrix):
        self.init_prob = init_prob
        self.trans_matrix = trans_matrix

    def get_page_rank_values(self, time_steps=None, set_init_prob=None, set_prob_trans_matrix=None):
        init_prob = self.init_prob if set_init_prob is None else set_init_prob
        trans_matrix = self.trans_matrix if set_prob_trans_matrix is None else set_prob_trans_matrix
        for _ in range(0, time_steps):
            init_prob = trans_matrix.dot(init_prob)
            print(np.abs(init_prob).shape)
            init_prob = init_prob / np.max(np.abs(init_prob))
        return init_prob / np.sum(init_prob)

In [8]:
trans_matrix=0.85*np.asarray([
    [0.,0.5,0.0,0],
    [1.0/3,0,0,1.0/2],
    [1.0/3,0,0,1.0/2],
    [1.0/3,1.0/2,0,0],
])+0.15*np.ones(shape=(4,4))/4.0
pr=PageRank(init_prob=init_prob,trans_matrix=trans_matrix)
pr.get_page_rank_values(10)

(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)


array([[0.1960504],
       [0.2679832],
       [0.2679832],
       [0.2679832]])

In [9]:
#检验下其他step
pr.get_page_rank_values(100)

(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)
(4, 1)


array([[0.19605034],
       [0.26798322],
       [0.26798322],
       [0.26798322]])